In [5]:
!pip install dtaidistance

In [6]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from collections import defaultdict
from tqdm import tqdm 
from dtaidistance import dtw

tqdm.pandas()

In [7]:
df_pred = pd.read_csv("../Data/DTW_df_pred.csv").drop("Unnamed: 0", axis=1)

In [8]:
df_pred.head()

,candidate_id,job_order,function_id,isco_code4
0,84556,0,1236.0,5131.0
1,84556,1,1065.0,9412.0
2,84556,2,1236.0,5131.0
3,84556,3,2038.0,9329.0
4,84556,4,2038.0,9329.0


In [9]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [10]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 468031/468031 [00:11<00:00, 39015.20it/s]


In [11]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1)]

In [45]:
results = defaultdict(lambda: defaultdict(int))

try:
    for i, current in tqdm(enumerate(career_paths.iteritems()), total=len(career_paths)):
        candidate, career = current
        career = career[:-1]

        for j in career_paths.iteritems():
            candidate2, career2 = j

            if candidate != candidate2:
                results[candidate][candidate2] = dtw.distance(career, 
                                                              career2, 
                                                              use_c=True, 
                                                              use_pruning=True)
                        
except KeyboardInterrupt:
    pass

  0%|          | 154/266487 [05:03<145:58:56,  1.97s/it]


In [46]:
best_matches = {}

for i in results:
    if len(results[i]):
        best_matches[i] = sorted(results[i], key=lambda x: results[i][x])

In [47]:
def knn_classifier(best_matches, career_paths, k=1):
    
    predictions = []
    
    for candidate in best_matches.keys():
        correct = career_paths[candidate][-1]
                
        neighbors = [(career_paths[neighbor][-1], i + 1) for i, neighbor in enumerate(best_matches[candidate][:k])]
        
        weights = defaultdict(int)
        
        for job, weight in neighbors:
            weights[job] += 1/np.log(1 + weight)
            
        predictions.append(correct == max(weights, key=lambda x: weights[x]))
        
    return np.mean(predictions)

In [49]:
results = defaultdict(list)

for k in range(1, 20):
    results["k"].append(k)
    results["accuracy"].append(knn_classifier(best_matches, career_paths, k=k))
    
pd.DataFrame(results).set_index("k")

,accuracy
k,
1,0.354839
2,0.354839
3,0.380645
4,0.387097
5,0.380645
6,0.374194
7,0.387097
8,0.380645
9,0.393548
